In [164]:
import os
from os.path import join
import copy
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

In [165]:
data  = join('C:', 'diabetes.csv')

In [166]:
data = pd.read_csv(data)
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [167]:
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

In [168]:
label = data['Outcome']
data = data[features]

In [169]:
data = data.replace(0,np.nan)

In [170]:
#train, valid(80%)  와 test(20%) 분리  (stratify=label)

In [171]:
from sklearn.model_selection import train_test_split
x_data, x_test, y_data, y_test = train_test_split(data,label, test_size=0.2,random_state=2021,shuffle=True,stratify=label)

In [172]:
#결측치를 평균으로 대체

In [173]:
x_data = x_data.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

In [174]:
x_mean = np.mean(x_data[features],axis=0)

In [175]:
for c in features:
    x_data.loc[pd.isna(x_data[c]),c] = x_mean[c]
    x_test.loc[pd.isna(x_test[c]),c] = x_mean[c]

In [176]:
#결측치 확인

In [177]:
pd.isna(x_data).sum(), pd.isna(x_test).sum()

(Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 dtype: int64,
 Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 dtype: int64)

In [178]:
x_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1.000000,128.0,98.000000,41.000000,58.000000,32.000000,1.321,33.0
1,9.000000,130.0,70.000000,29.184332,156.865385,34.200000,0.652,45.0
2,9.000000,57.0,80.000000,37.000000,156.865385,32.800000,0.096,41.0
3,9.000000,170.0,74.000000,31.000000,156.865385,44.000000,0.403,43.0
4,7.000000,119.0,71.946918,29.184332,156.865385,25.200000,0.209,37.0
...,...,...,...,...,...,...,...,...
609,3.000000,80.0,71.946918,29.184332,156.865385,32.731126,0.174,22.0
610,4.469349,121.0,66.000000,30.000000,165.000000,34.300000,0.203,33.0
611,4.469349,93.0,60.000000,29.184332,156.865385,35.300000,0.263,25.0
612,3.000000,84.0,68.000000,30.000000,106.000000,31.900000,0.591,25.0


In [179]:
y_data = y_data.reset_index(drop=True)

In [180]:
y_data

0      1
1      1
2      0
3      1
4      0
      ..
609    0
610    1
611    0
612    0
613    0
Name: Outcome, Length: 614, dtype: int64

In [161]:
#기본

In [181]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, 
                                                      test_size=0.3,
                                                      random_state=2021,
                                                      shuffle=True,
                                                      stratify=y_data)

In [182]:
x_train_mean = np.mean(x_train[features], axis=0)
x_train_std  = np.std(x_train[features], axis=0)

In [183]:
x_train.loc[:, features] = (x_train[features] - x_train_mean) / x_train_std 
x_valid.loc[:, features] = (x_valid[features] - x_train_mean) / x_train_std

In [184]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()

In [185]:
lgb.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	training's binary_logloss: 0.0666379	valid_1's auc: 0.807593	valid_1's binary_logloss: 0.628018
Did not meet early stopping. Best iteration is:
[78]	training's auc: 1	training's binary_logloss: 0.098913	valid_1's auc: 0.807335	valid_1's binary_logloss: 0.588787


LGBMClassifier()

In [186]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [189]:
lgb.feature_importances_

array([131, 231, 132,  92,  71, 233, 237, 190])

In [190]:
importances = pd.DataFrame({'feature': x_train.columns, 'importance': lgb.feature_importances_})
importances = importances.sort_values(by='importance',ascending=False)
importances.reset_index(drop=True, inplace=True)
importances

,feature,importance
0,DiabetesPedigreeFunction,237
1,BMI,233
2,Glucose,231
3,Age,190
4,BloodPressure,132
5,Pregnancies,131
6,SkinThickness,92
7,Insulin,71


In [191]:
#하이퍼파라미터 튜닝

In [192]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

In [193]:
# parameter 별로 search할 범위를 설정. 
bayesian_params = {
    'max_depth': (6, 16), 
    'num_leaves': (24, 64), 
    'min_child_samples': (10, 200), 
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_bin':(10, 500)
}

In [194]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree,max_bin):
    params = {
        "n_estimators":500, "learning_rate":0.01,
        'max_depth': int(round(max_depth)), #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_bin':  max(int(round(max_bin)),10)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)
    valid_proba = lgb_model.predict_proba(x_valid)[:, 1]
    roc_auc = roc_auc_score(y_valid, valid_proba)
    
    return roc_auc   

In [195]:
# BayesianOptimization객체를 수행할 함수와 search할 parameter 범위를 설정하여 생성. 
lgbBO = BayesianOptimization(lgb_roc_eval,bayesian_params , random_state=2021)
# 함수 반환값이 최대가 되는 입력값 유추를 위한 iteration 수행. 
lgbBO.maximize(init_points=5, n_iter=25) #초기 random iter 횟수 / drill down

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.5	training's binary_logloss: 0.687876	valid_1's auc: 0.5	valid_1's binary_logloss: 0.683278
Early stopping, best iteration is:
[1]	training's auc: 0.5	training's binary_logloss: 0.687876	valid_1's auc: 0.5	valid_1's binary_logloss: 0.683278
|  1        |  0.5      |  0.803    |  369.4    |  7.389    |  69.41    |  49.86    |  29.13    |  0.5895   |
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.906511	training's binary_logloss: 0.452522	valid_1's auc: 0.80288	valid_1's binary_logloss: 0.521774
[200]	training's auc: 0.935305	training's binary_logloss: 0.366618	valid_1's auc: 0.819861	valid_1's binary_logloss: 0.489633
[300]	training's auc: 0.951302

In [196]:
# BayesianOptimization객체의 res는 iteration 수행 시마다 모든 함수 반환결과와 그때의 파라미터 결과값을 가지고 있음. 
lgbBO.res

[{'target': 0.5,
  'params': {'colsample_bytree': 0.8029891394037023,
   'max_bin': 369.3509869848141,
   'max_depth': 7.389471567283987,
   'min_child_samples': 69.40788593239051,
   'min_child_weight': 49.864920785675615,
   'num_leaves': 29.126495016758426,
   'subsample': 0.589496552975094}},
 {'target': 0.8203770661157025,
  'params': {'colsample_bytree': 0.8764627143880469,
   'max_bin': 334.4586520116717,
   'max_depth': 13.843101321411226,
   'min_child_samples': 28.409935225723554,
   'min_child_weight': 3.869992969313764,
   'num_leaves': 62.495839608415665,
   'subsample': 0.8082787219472871}},
 {'target': 0.7647210743801653,
  'params': {'colsample_bytree': 0.5433149806242736,
   'max_bin': 285.02345762509634,
   'max_depth': 12.165247086179901,
   'min_child_samples': 193.1301743463701,
   'min_child_weight': 29.140910428278865,
   'num_leaves': 38.84643392631126,
   'subsample': 0.7260726218947822}},
 {'target': 0.7925490702479339,
  'params': {'colsample_bytree': 0.60092

In [197]:
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmax(np.array(target_list)))

[0.5, 0.8203770661157025, 0.7647210743801653, 0.7925490702479339, 0.7627195247933883, 0.7896435950413223, 0.8110795454545454, 0.819085743801653, 0.7012525826446281, 0.8288997933884298, 0.8211518595041322, 0.5, 0.8225723140495868, 0.8307076446280991, 0.7563920454545454, 0.8270919421487603, 0.8175361570247933, 0.8070764462809917, 0.8216683884297521, 0.5, 0.7596849173553719, 0.759426652892562, 0.821926652892562, 0.8212809917355373, 0.8175361570247933, 0.5, 0.7975852272727273, 0.7596849173553719, 0.5, 0.8326446280991736]
maximum target index: 29


In [198]:
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.8326446280991736, 'params': {'colsample_bytree': 0.5, 'max_bin': 244.59637674642823, 'max_depth': 16.0, 'min_child_samples': 35.657663580429876, 'min_child_weight': 1.0, 'num_leaves': 28.33735096230287, 'subsample': 1.0}}


In [223]:
params = {
        "n_estimators":500, "learning_rate":0.01,
        'max_depth': 16, #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': 28, 
        'min_child_samples': 35,
        'min_child_weight': 1.0,
        'subsample': 1.0, 
        'colsample_bytree': 0.5,
        'max_bin': 244
    }
model = LGBMClassifier(**params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.888754	training's binary_logloss: 0.479575	valid_1's auc: 0.873214	valid_1's binary_logloss: 0.483405
[200]	training's auc: 0.912209	training's binary_logloss: 0.409394	valid_1's auc: 0.88244	valid_1's binary_logloss: 0.432561
Early stopping, best iteration is:
[192]	training's auc: 0.910738	training's binary_logloss: 0.413816	valid_1's auc: 0.883929	valid_1's binary_logloss: 0.434394


LGBMClassifier(colsample_bytree=0.5, learning_rate=0.01, max_bin=244,
               max_depth=16, min_child_samples=35, min_child_weight=1.0,
               n_estimators=500, num_leaves=28)

In [202]:
#OOF 적용

In [210]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)

In [211]:
def preprocess(x_train, x_valid):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    
    x_train[features] = scaler.fit_transform(x_train[features])
    x_valid[features] = scaler.fit_transform(x_valid[features])
    
   
    
    return tmp_x_train, tmp_x_valid

In [218]:
accuracy_scores = list()
f1_scores = list()
auc_scores = list()


for i, (trn_idx, val_idx) in enumerate(skf.split(x_data, y_data)):
    x_train, y_train = x_data.iloc[trn_idx, :], y_data[trn_idx]
    x_valid, y_valid = x_data.iloc[val_idx, :], y_data[val_idx]
    
    # 전처리
    x_train, x_valid= preprocess(x_train, x_valid)
    
    params = {
        "n_estimators":500, "learning_rate":0.01,
        'max_depth': 6, #  호출 시 실수형 값이 들어오므로 정수형 하이퍼 파라미터는 정수형으로 변경 
        'num_leaves': 64, 
        'min_child_samples': 10,
        'min_child_weight': 1.0,
        'subsample': 0.63, 
        'colsample_bytree': 1.0,
        'max_bin': 150
    }
    model = LGBMClassifier(**params)
    
    model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], eval_metric= 'auc', verbose= 100, 
                early_stopping_rounds= 100)

    # 훈련, 검증 데이터 Log Loss 확인
    val_accuracy = accuracy_score(y_valid, model.predict(x_valid))
    f1_accuracy = f1_score(y_valid, model.predict(x_valid))
    auc_accuracy = roc_auc_score(y_valid, model.predict(x_valid))
    accuracy_scores.append(val_accuracy)
    f1_scores.append(f1_accuracy)
    auc_scores.append(auc_accuracy)
    
    
   
    

# 교차 검증 Log Loss 평균 계산하기
print('accuracy Score : {:.4f}'.format(np.mean(accuracy_scores)))
print('f1 Score : {:.4f}'.format(np.mean(f1_scores)))
print('auc Score : {:.4f}'.format(np.mean(auc_scores)))

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.96017	training's binary_logloss: 0.392983	valid_1's auc: 0.826744	valid_1's binary_logloss: 0.49936
[200]	training's auc: 0.977632	training's binary_logloss: 0.296646	valid_1's auc: 0.833721	valid_1's binary_logloss: 0.480835
Early stopping, best iteration is:
[180]	training's auc: 0.975548	training's binary_logloss: 0.310678	valid_1's auc: 0.834884	valid_1's binary_logloss: 0.481142
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965662	training's binary_logloss: 0.382727	valid_1's auc: 0.812355	valid_1's binary_logloss: 0.519209
[200]	training's auc: 0.980044	training's binary_logloss: 0.291558	valid_1's auc: 0.824419	valid_1's binary_logloss: 0.498203
[300]	training's auc: 0.987409	training's binary_logloss: 0.248399	valid_1's auc: 0.82093	valid_1's binary_logloss: 0.507987
Early stopping, best iteration is:
[205]	training's auc: 0.980537	training's binary_logl